## Homework 6: Digit detection with TensorFlow

## Nicolás Echeverri Rojas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.5.0'

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten

## 0) Primer modelo

### 0.1) Carga de datos

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
X_train, X_test,y_train,y_test=train_test_split(featureMatrix,target,test_size=0.2)

NameError: name 'featureMatrix' is not defined

In [ ]:
X_train.shape

### 0.2) Creación del modelo

In [ ]:
tf.random.set_seed(1234)

model = Sequential([
                  Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
                  Dense(units=100,activation='relu',name='layer_1'),
                  Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                 ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
myhistory=model.fit(X_train, y_train,  epochs=10, batch_size=32, validation_split=0.1, verbose=2)

### 0.3 Evaluación del modelo

In [ ]:
model.evaluate( X_train,y_train)

In [ ]:
model.evaluate( X_test,y_test)

Se observa que el primer modelo alcanza una precisión bastante buena puesto que en los datos de train alcanza una precisión de 0.953, en los de validación 0.953 y en los de test de 0.942 sin embargo se va a intentar estos valores

## Aumentando el Performace del modelo de red neuronal

### 1) Normalización de los datos

Primero se realiza una normalización de los datos de tal manera que se evite vanish gradient o exploding gradient 


### 1.1) Normalización a Gaussiana estandar

In [ ]:
from sklearn import preprocessing

In [ ]:
X_train_reshaped=X_train.reshape(len(X_train),28*28)
X_test_reshaped=X_test.reshape(len(X_test),28*28)

In [ ]:
X_train_normalized=preprocessing.StandardScaler().fit(X_train_reshaped).transform( X_train_reshaped)
X_test_normalized=preprocessing.StandardScaler().fit(X_test_reshaped).transform( X_test_reshaped)

In [ ]:
tf.random.set_seed(1234)
model = Sequential([
                  InputLayer(input_shape=(28*28,)),
                  Dense(units=100,activation='relu',name='layer_1'),
                  Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                 ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_normalized, y_train,  epochs=10, batch_size=32, validation_split=0.1, verbose=2)

In [ ]:
model.evaluate( X_test_normalized,y_test)

Al normalizar los datos las predicciones del modelo mejoran en gran medida pues alcanza una precisión del 0.993 en los datos de entranamiento, 0.968 en los datos de validación y 0.967 en los datos de test

### 1.2) Normalización por escalamiento

In [ ]:
maxValue=np.max(X_train)

In [ ]:
X_train_scaled, X_test_scaled=X_train/maxValue, X_test/maxValue

In [ ]:
model = Sequential([
                  Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
                  Dense(units=100,activation='relu',name='layer_1'),
                  Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                 ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

myHistory=model.fit(X_train_scaled, y_train,  epochs=10, batch_size=32, validation_split=0.1, verbose=2)

In [ ]:
myhistory.history['val_accuracy'][-1],myhistory.history['accuracy'][-1]

In [ ]:
myhistory.history.keys()

In [ ]:
model.evaluate( X_test_scaled,y_test)

Se obseva que este modelo posee una precisión de 0.9932 en datos de train,0.9772 en datos de validación y 0.9772 en datos de test, de tal manera que este escalamiento es el que mejor resultados ha arrojado

## 2) Variando hiperparámetros del modelo

In [ ]:
def my_model(X_train, y_train,X_test, y_test, optimizerName,numberEpoch=10, plot=False,verbose=0):
        
        model = Sequential([
                  Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
                  Dense(units=100,activation='relu',name='layer_hidden'),
                  Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                 ])
        
        model.compile(optimizer=optimizerName,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
        
        myhistory=model.fit(X_train, y_train,  epochs=numberEpoch, batch_size=32, validation_split=0.1, verbose=verbose)
        
        TrainAccuracy=myhistory.history['accuracy'][-1]
        LossTrain=myhistory.history['loss'][-1]
        
        ValidationAccuracy=myhistory.history['val_accuracy'][-1]
        LossValidation=myhistory.history['val_loss'][-1]
        
        LossTest,AccuracyTest=model.evaluate( X_test,y_test)
        
        
        if plot is True:
            xvals=range(numberEpoch)
            fig,ax=plt.subplots(1,1)
            ax.plot(xvals, myhistory.history['val_accuracy'],label='validation set accuracy')
            ax.plot(xvals, myhistory.history['accuracy'],label='train set accuracy')
            ax.set_xlabel('epochs')
            ax.set_ylabel('accuracy')
            ax.legend()
        
        return [ [LossTrain,TrainAccuracy],
                [LossValidation,ValidationAccuracy],
                [LossTest,AccuracyTest] ]
        


### 2.1) Probando optimizadores

### 2.1.1) adam

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='adam',plot=True,verbose=2)

### 2.1.2) SGD

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='SGD',plot=True)


### 2.1.3) RMSprop

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',plot=True)


### 2.1.3) Adagrad

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='Adagrad',plot=True)

### 2.1.4) Adadelta

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='Adadelta',plot=True)

### 2.1.5) Adamax

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='Adamax',plot=True)

Se observa que el mejor optimizador es RMSprop puesto que posee la mayor precisión en el conjunto de test pues alcanza a poseer 0.9763

En la siguiente tabla se resumen los resultados

| Optimizador | Train accuracy | Validation accuracy | Test accuracy |
| --- | --- | --- | --- |
| adam | 0.9940 | 0.9761 |  0.9722  |
| SGD | 0.9486 | 0.9599 |    0.9478 | 
| RMSprop | 0.9894 | 0.9750 |   0.9750 |
| Adagrad | 0.8998 | 0.9231 |  0.9070  |
| Adadelta | 0.7555 | 0.79983 |0.7835    |
| Adamax | 0.9773 | 0.97516 |   0.9715  |




### 2.2) Variando número de épocas del mejor optimizador

### 2.2.1) 20 épocas

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=20,plot=True)

### 2.2.2) 25 épocas

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=25,plot=True)

### 2.2.3) 40 épocas

In [ ]:
my_model(X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=40,plot=True)

Se observa que en alrededor de 8 épocas se empieza a estabilizar la precisión en el conjunto de validación por tanto se opta por entrenar el modelo en 12 épocas ya que no se observa un incremento sustancial en la precisión al incrementar más estas

### 2.3) Variando la arquitectura de la red

### 2.3.1) Variando número de neuronas

In [ ]:
def my_model2(model,X_train, y_train,X_test, y_test, optimizerName,numberEpoch=10, plot=False,verbose=0, lossF='sparse_categorical_crossentropy'):
        tf.random.set_seed(1234)
        model.compile(optimizer=optimizerName,
              loss=lossF,
              metrics=['accuracy'])
        
        myhistory=model.fit(X_train, y_train,  epochs=numberEpoch, batch_size=32, validation_split=0.1, verbose=verbose)
        
        TrainAccuracy=myhistory.history['accuracy'][-1]
        LossTrain=myhistory.history['loss'][-1]
        
        ValidationAccuracy=myhistory.history['val_accuracy'][-1]
        LossValidation=myhistory.history['val_loss'][-1]
        
        LossTest,AccuracyTest=model.evaluate( X_test,y_test)
        
        
        if plot is True:
            xvals=range(numberEpoch)
            fig,ax=plt.subplots(1,1)
            ax.plot(xvals, myhistory.history['val_accuracy'],label='validation set accuracy')
            ax.plot(xvals, myhistory.history['accuracy'],label='train set accuracy')
            ax.set_xlabel('epochs')
            ax.set_ylabel('accuracy')
            ax.legend()
        
        return [ [LossTrain,TrainAccuracy],
                [LossValidation,ValidationAccuracy],
                [LossTest,AccuracyTest] ]

### 2.3.1.1)  50 neuronas

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=50,activation='relu',name='layer_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.1.2)   150 neuronas

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=150,activation='relu',name='layer_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.1.3)   200 neuronas

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=200,activation='relu',name='layer_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

Se observa que al aumentar el número de neuronas también aumenta la precisión puesto que usando 200 neuronas se alcanza una precisión de 0.979 en los datos de test, sin embargo el tiempo de cómputo ha empezado a ser considerable

En la siguiente tabla se resumen los resultados

| número neuronas | Train accuracy | Validation accuracy | Test accuracy |
| --- | --- | --- | --- |
| 50 | 0.9839 | 0.9743 |   0.9708  |
| 100 | 0.9894 | 0.9750 |   0.9750 |
| 150 | 0.9943 |  0.9778 |    0.9783 | 
| 200 | 0.9951 | 0.9819 |  0.9793  |

### 2.3.2) Variando número de capas y número de neuronas

### 2.3.2.1) modelo 1: 50-50

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=50,activation='relu',name='layer1_hidden'),
        Dense(units=50,activation='relu',name='layer2_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.2.2) modelo 2: 100-50

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=100,activation='relu',name='layer1_hidden'),
        Dense(units=50,activation='relu',name='layer2_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

Se observa que el modelo de dos capas, la primera de 100 y la segunda de 50 da un resultado de 0.9775, en comparación con el modelo de una capa de 150 neuronas con un resultado de  0.9783, de tal manera que este segundo modelo aunque tienen el mismo número de neuronas, posee mayor precisión

### 2.3.2.3) modelo 3: 100-50-25

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=100,activation='relu',name='layer1_hidden'),
        Dense(units=50,activation='relu',name='layer2_hidden'),
        Dense(units=25,activation='relu',name='layer3_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.2.4) modelo 4: 100-100

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=100,activation='relu',name='layer1_hidden'),
        Dense(units=100,activation='relu',name='layer2_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

el modelo de dos capas de cada uno de de 100 neuronas posee una precisión de 0.9782 en comparación con el modelo de 200 neuronas en una sola capa que da como resultado 0.9793, de tal manera que este segundo modelo posee una mayor precisión

### 2.3.2.5) modelo 5: 200-150

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=200,activation='relu',name='layer1_hidden'),
        Dense(units=150,activation='relu',name='layer2_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.2.6) modelo 6: 200-200

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=200,activation='relu',name='layer1_hidden'),
        Dense(units=200,activation='relu',name='layer2_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

### 2.3.2.7) modelo 7: 200-150-50

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=200,activation='relu',name='layer1_hidden'),
        Dense(units=150,activation='relu',name='layer2_hidden'),
        Dense(units=50,activation='relu',name='layer3_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train,X_test_scaled, y_test, optimizerName='RMSprop',numberEpoch=12,plot=True)

Se observa que aunque se haya incrementado el número de capas y de neuronas, el desemepeño de este sigue siendo menor al del una capa cocon 200 neuronas pues este alcanzó 0.979 en precisión en los datos de test, por tanto, se sigue usando este modelo para las siguientes variaciones de hiperparámetros

En la siguiente tabla se resumen los resultados

| Arquitectura | Train accuracy | Validation accuracy | Test accuracy |
| --- | --- | --- | --- |
| Arquitectura 0: 200 | 0.9951 | 0.9819 |  0.9793  |
| Arquitectura 1: 50-50 | 0.9879 | 0.9773 |   0.9732 |
| Arquitectura 2:100-50 | 0.9927 | 0.9796 |   0.9775 |
| Arquitectura 3: 100-50-25| 0.9912 |  0.9791 |  0.9753 | 
| Arquitectura 4: 100-100| 0.9935 | 0.97983 |  0.9782 |
| Arquitectura 5: 200-150| 0.9950 | 0.9801 |  0.9789  |
| Arquitectura 6: 200-200| 0.9946 | 0.9785 |  0.9768  |
| Arquitectura 7: 200-150-50| 0.9951 | 0.9819 |  0.9781  |

## 2.4) Usando one-hot encoding para la clasificación

In [ ]:
from keras.utils.np_utils import to_categorical   
y_train_tf_one_hot = to_categorical(y_train, num_classes=10)
y_test_tf_one_hot = to_categorical(y_test, num_classes=10)

In [ ]:
model = Sequential([
        Flatten(input_shape=(28,28)), #Para aplanar el arreglo de 28x28
        Dense(units=200,activation='relu',name='layer1_hidden'),
        Dense(units=10,activation='softmax',name='output_layer') #10 neuronas porque se tiene 10 categorias
                ])
my_model2(model,X_train_scaled, y_train_tf_one_hot ,X_test_scaled, y_test_tf_one_hot, 
          optimizerName='RMSprop',numberEpoch=12,lossF='categorical_crossentropy',plot=True)

Se observa que al utilizar codificación one-hot encoding se aumenta un poco la precisión (0.04%) en los datos de test

| modelo | Train accuracy | Validation accuracy | Test accuracy |
| --- | --- | --- | --- |
| Sin one-hot | 0.9951 | 0.9819 |  0.9793  |
| con one-hot | 0.9952 | 0.9816 |  0.9797 |

## 3) Conclusiones

Luego de explorar el espacio de los hiperparámetros para realizar la clasificación de dígitos se encontró que el mejor modelo prediciendo los datos posee las características:

- Escalamiento de datos: Escalamiento usando valor máximo de los datos

- Codificación: one-hot-encoding

- optimizador: RMSprop

- arquitectura: MLP, 1 hidden layer de 200 neuronas

- número épocas: 12

Dando como resultado:


| Model | Train accuracy | Validation accuracy | Test accuracy |
| --- | --- | --- | --- |
| First model | 0.9554 |  0.9527 |  0.9419  |
| Best model | 0.9952 | 0.9816 |  0.9797 |
|      Improvement      |    4%    |   3%     |     4%    |

De tal manera que se mejoró el modelo un 4% realizando la variación de los hiperparámetros.

En ningún modelo de los presentado se presentó underfitting o overfitting puesto que el error en los datos de validación jamás empezó a incrementar.
